In [1]:
import json

# import the smartpynector module
from NBs import smartpynector as sp

In [2]:
# Defining constants
# Knowledge engine REST API URL
KNOWLEDGE_ENGINE_URL = "http://127.0.0.1:8280/rest"
SUBSCRIBER_ID = "http://127.0.0.1:8000/subscriber"

In [3]:
# Defining the smart connector
smart_connector = sp.SmartConnector(
    knowledgeBaseId=SUBSCRIBER_ID,
    knowledgeBaseName="Subscriber2",
    knowledgeBaseDescription="Subscriber2",
    reasonerEnabled=False,
)

In [4]:
sp.create_smart_connector(smart_connector, KNOWLEDGE_ENGINE_URL)

2023-05-03 18:37:03 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-03 18:37:04 DEBUG http://127.0.0.1:8280 "POST /rest/sc HTTP/1.1" 200 0
2023-05-03 18:37:04 INFO Request successful
2023-05-03 18:37:04 DEBUG 


<Response [200]>

In [5]:
GRAPH_PATTERN= '''?meas rdf:type saref:Measurement .
         ?meas saref:hasValue ?temp .
         ?meas saref:isMeasuredIn saref:TemperatureUnit .
         ?meas saref:hasTimestamp ?timestamp .
         ?meas saref:isMeasurementOf ?room_id .
         ?meas saref:relatesToProperty saref:Temperature .
         ?meas saref:measurementMadeBy ?device_id .'''

In [9]:
ask_ki = sp.register_knowledge_interaction(
    knowledge_interaction_type="AskKnowledgeInteraction",
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    prefixes={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    },
    graph_pattern=GRAPH_PATTERN,
    headers={
        "Content-Type": "application/json",
        "knowledge-Base-Id": SUBSCRIBER_ID,
    },
)
ask_ki

2023-05-03 18:38:28 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-03 18:38:28 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ki HTTP/1.1" 200 110
2023-05-03 18:38:28 INFO Request successful
2023-05-03 18:38:28 DEBUG {"knowledgeInteractionId":"http://127.0.0.1:8000/subscriber/interaction/5727cb2a-043e-47e2-842e-7377a77747e1"}


{'knowledgeInteractionId': 'http://127.0.0.1:8000/subscriber/interaction/5727cb2a-043e-47e2-842e-7377a77747e1',
 'prefixes': {'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
  'saref': 'https://w3id.org/saref#'}}

In [10]:
binding_set = sp.perform_ask_knowledge_interaction(
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    knowledge_base_id=SUBSCRIBER_ID,
    knowledge_interaction_id=ask_ki["knowledgeInteractionId"],
    ask_data=[]
).json()["bindingSet"]

2023-05-03 18:38:28 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-03 18:38:29 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ask HTTP/1.1" 200 895
2023-05-03 18:38:29 INFO Request successful
2023-05-03 18:38:29 DEBUG {"bindingSet":[{"room_id":"\"http://127.0.0.1:8000/thermostat/rooms/1\"","temp":"20","device_id":"\"http://127.0.0.1:8000/thermostat/devices/1\"","meas":"<http://127.0.0.1:8000/thermostat/measurements/e6cf71b8-327f-47a5-9569-c201742a5814>","timestamp":"\"2023-05-03T18:38:24+02:00\""}],"exchangeInfo":[{"initiator":"knowledgeBase","knowledgeBaseId":"http://127.0.0.1:8000/thermostat","knowledgeInteractionId":"http://127.0.0.1:8000/thermostat/interaction/462628e6-739b-4dd2-8a18-b42b7b9d5234","exchangeStart":"2023-05-03T16:38:29.018+00:00","exchangeEnd":"2023-05-03T16:38:29.025+00:00","status":"SUCCEEDED","bindingSet":[{"room_id":"\"http://127.0.0.1:8000/thermostat/rooms/1\"","temp":"20","device_id":"\"http://127.0.0.1:8000/thermostat/devices/1\"","meas":"<http://1

In [11]:
READ_URL = "https://graphdb.odissei.nl/repositories/MateuszTest"
WRITE_URL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

sp.store_data_in_graphdb(
    graph_pattern=GRAPH_PATTERN,
    binding_set=binding_set,
    prefixes=ask_ki["prefixes"],
    read_url=READ_URL,
    write_url=WRITE_URL,
)

Ignoring 500 server error
